In [2]:
import os
import json
import re
from typing import List
import jsonlines
from tqdm import tqdm


In [3]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.nli
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/snli-roberta-large-2020.04.30.tar.gz", predictor_name="textual-entailment")

In [9]:
def entailment_score(hypothesis:List, premises:List, thredshold:float)->float:
    semantic_score, valid = 0, 0
    assert len(hypothesis) == len(premises), print("the length of hypothesis should be the same")
    for hyp, pre in zip(hypothesis, premises):
        score = predictor.predict(hypothesis=hyp, premise=pre)['probs'][0]
        semantic_score += score if score > thredshold else 0.0
        valid +=1 if score > thredshold else 0
    return semantic_score/valid if valid else 0.0

In [5]:
def read_json(file):
    with open(file) as f:
        data = json.load(f)
    return data

In [22]:
prediction = read_json("result_output_updated.json")
prediction = prediction[:10] # for testing


In [23]:
threshold = 0.5 # set the threshold of nli_score be 0.5
model_evaluation_score= 0 
for pred in tqdm(prediction):
    model_pred = pred['pred']
    ref = pred['ref']
    label_score = pred['score']
    pred_score = 0
    most_similar_label = ""
    for label, premises in ref.items():
        if premises: # if the premises is not empty
            hypothesis = [prem.replace(label, model_pred) for prem in premises]
            nli_score=entailment_score(hypothesis, premises, threshold) 
            if nli_score > pred_score:
                pred_score = nli_score
                most_similar_label = label
    # print(most_similar_label,pred_score)
    if pred_score:
        model_evaluation_score+=pred_score * label_score[most_similar_label] # multiply the nli score with the original score. 
print("the semantic evaluation score for the model is {:.2f}".format(model_evaluation_score/len(prediction)) )

100%|██████████| 10/10 [00:20<00:00,  2.01s/it]

the semantic evaluation score for the model is 0.49
